In [14]:
import sys
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import collections
import scipy.stats
import statistics
from Bio import SeqIO
from itertools import product
from scipy.fftpack import fft, ifft
import math 

In [6]:

def headerNAC(k, foutput):
    global listTotal, caracteres
    listTotal = []
    file = open(foutput, 'a')
    file.write('%s,' % ('nameseq'))
    permsList = [''.join(str(i) for i in x) for x in product(caracteres, repeat=k)]
    # print (permsList)
    for perm in permsList:
        # print (perm)
        listTotal.append(perm)
        file.write('%s,' % (str(perm)))
    file.write('label')
    file.write('\n')
    return


def chunks(seq, win, step):
    seqlen = len(seq)
    for i in range(0, seqlen, step):
        j = seqlen if i+win>seqlen else i+win
        yield seq[i:j]
        if j==seqlen: break
    return        
    

def chunks_two(seq, win):
    seqlen = len(seq)
    for i in range(seqlen):
        j = seqlen if i+win>seqlen else i+win
        yield seq[i:j]
        if j==seqlen: break
    return

            
def file_record(name_seq, foutput, labelDataset):
    file = open(foutput, 'a')
    file.write('%s,' % (name_seq))
    for x in probabilities:
        # print (x)
        file.write('%s,' % (str(x[1])))
    file.write(labelDataset)
    file.write('\n')
    print ('Recorded Sequence: %s' % (name_seq))
    return
    

# Nucleic acid composition (NAC), Di-nucleotide composition (DNC), Tri-nucleotide composition (TNC)
def nacSeq(k, foutput, finput, labelDataset):
    global probabilities
    headerNAC(k, foutput)
    for seq_record in SeqIO.parse(finput, 'fasta'):
        seq = seq_record.seq
        seq = seq.upper()	
        name_seq = seq_record.name
        probabilities = []
        kmer = {}
        totalWindows = (len(seq) - k) + 1 # (L - k + 1)
        permsList = [''.join(str(i) for i in x) for x in product(caracteres, repeat=k)]
        for key in permsList:
            kmer[key] = 0
        kmer = collections.OrderedDict(sorted(kmer.items()))
        for subseq in chunks_two(seq, k):
            if subseq in kmer:
                # print(subseq)
                kmer[subseq] = kmer[subseq] + 1
            else:
                kmer[subseq] = 1
        for key, value in kmer.items():
            # print (key)
            # print (value)
            probabilities.append([str(key), value/totalWindows])
        file_record(name_seq, foutput, labelDataset)
    return


def ExtractionTechniques(finput, foutput, tech):
    global listTotal, caracteres, complement
    labelDataset = 'DNA'
    # tech = str(args.type)
    seq = 1
    stepw = 1
    if seq == 1:
        caracteres = ['A', 'C', 'G', 'T']
        complement = {'A': 'T','C': 'G','G': 'C','T': 'A'}
    else:
        caracteres = ['A', 'C', 'G', 'U']
        complement = {'A': 'U','C': 'G','G': 'C','U': 'A'}
    if tech == 'NAC' or tech == 'nac':
        nacSeq(1, foutput, finput, labelDataset)
    elif tech == 'DNC' or tech == 'dnc':
        nacSeq(2, foutput, finput, labelDataset)
    elif tech == 'TNC' or tech == 'tnc':
        nacSeq(3, foutput, finput, labelDataset)
#############################################################################

In [12]:
def header(foutput):
	dataset = open(foutput, 'a')
	dataset.write("nameseq,average,median,maximum,minimum,peak,"
                  + "none_levated_peak,sample_standard_deviation,population_standard_deviation,"
                  + "percentile15,percentile25,percentile50,percentile75,amplitude,"
                  + "variance,interquartile_range,semi_interquartile_range,"
                  + "coefficient_of_variation,skewness,kurtosis,label")
	dataset.write("\n")
	return

        
def file_record(foutput, label_dataset):
    dataset = open(foutput, 'a')
    dataset.write("%s," % (str(name_seq)))
    for metric in features:
        dataset.write("%s," % (metric))
        # dataset.write("{0:.4f},".format(metric))
    dataset.write(label_dataset)
    dataset.write("\n")
    print("Sequence Analyzed!!")
    return


def feature_extraction():
    global features
    features = []
    average = sum(spectrum)/len(spectrum)
    features.append(average)
    ###################################
    median = np.median(spectrum)
    features.append(median)
	###################################
    maximum = np.max(spectrum)
    features.append(maximum)
    ###################################
    minimum = np.min(spectrum)
    features.append(minimum)
    ###################################
    peak = (len(spectrum)/3)/(average)
    features.append(peak)
    ###################################
    peak_two = (len(spectrumTwo)/3)/(np.mean(spectrumTwo))
    features.append(peak_two)
    ###################################
    standard_deviation = np.std(spectrum) # standard deviation
    features.append(standard_deviation)
    ###################################
    standard_deviation_pop = statistics.stdev(spectrum) # population sample standard deviation 
    features.append(standard_deviation_pop)
    ###################################
    percentile15 = np.percentile(spectrum, 15)
    features.append(percentile15)
    ###################################
    percentile25 = np.percentile(spectrum, 25)
    features.append(percentile25)
    ###################################
    percentile50 = np.percentile(spectrum, 50)
    features.append(percentile50)
    ###################################
    percentile75 = np.percentile(spectrum, 75)
    features.append(percentile75)
    ###################################
    amplitude = maximum - minimum
    features.append(amplitude)
    ###################################
    # mode = statistics.mode(spectrum)
    ###################################
    variance = statistics.variance(spectrum)
    features.append(variance)
    ###################################
    interquartile_range = np.percentile(spectrum, 75) - np.percentile(spectrum, 25)
    features.append(interquartile_range)
    ###################################
    semi_interquartile_range = (np.percentile(spectrum, 75) - np.percentile(spectrum, 25))/2 
    features.append(semi_interquartile_range)
    ###################################
    coefficient_of_variation = standard_deviation/average
    features.append(coefficient_of_variation)
    ###################################
    skewness = (3 * (average - median))/standard_deviation
    features.append(skewness)   
    ###################################
    kurtosis = (np.percentile(spectrum, 75) - np.percentile(spectrum, 25)) / (2 * (np.percentile(spectrum, 90) - np.percentile(spectrum, 10))) 
    features.append(kurtosis)
    ###################################
    return


def real_fourier(finput, foutput, label_dataset):
    header(foutput)
    global spectrum, spectrumTwo, name_seq
    for seq_record in SeqIO.parse(finput, "fasta"):
        seq = seq_record.seq
        seq = seq.upper()
        name_seq = seq_record.name
        spectrum = []
        spectrumTwo = []
        real = []
        for nucle in seq:
            if nucle == "T" or nucle == "U":
                real.append(1.5)
            elif nucle == "C":
                real.append(0.5)
            elif nucle == "A":
                real.append(-1.5)
            else:
                real.append(-0.5)
        FR = fft(real)
        for i in range(len(seq)):
            specTotal = (abs(FR[i])**2)
            specTwo = (abs(FR[i]))
            spectrum.append(specTotal)
            spectrumTwo.append(specTwo)
        feature_extraction()
        file_record(foutput, label_dataset)
    return


#############################################################################    
def FourierClass(finput, foutput, label_dataset):
    real_fourier(finput, foutput, label_dataset)
#############################################################################

In [16]:
def EntropyClass_header(foutput, ksize):
    file = open(foutput, 'a')
    file.write("nameseq,")
    for i in range(1, ksize+1):
        file.write("k" + str(i) + ",")
    file.write("label")
    file.write("\n")
    return


def EntropyClass_chunks_two(seq, win):
    seqlen = len(seq)
    for i in range(seqlen):
        j = seqlen if i+win>seqlen else i+win
        yield seq[i:j]
        if j==seqlen: break
    return

            
def EntropyClass_file_record(foutput, label_dataset):
    file = open(foutput, 'a')
    file.write("%s," % (name_seq))
    for data in information_entropy:
        file.write("%s," % (str(data)))
    file.write(label_dataset)
    file.write("\n")
    print ("Recorded Sequence!!!")
    return
    

def entropy_equation(ksize, finput, foutput, e, label_dataset):
    EntropyClass_header(foutput, ksize)
    global name_seq, information_entropy
    for seq_record in SeqIO.parse(finput, "fasta"):
        seq = seq_record.seq
        seq = seq.upper()
        name_seq = seq_record.name
        information_entropy = []
        for k in range(1, ksize+1):
            probabilities = []
            kmer = {}
            total_windows = (len(seq) - k) + 1 # (L - k + 1)
            for subseq in EntropyClass_chunks_two(seq, k):
                if subseq in kmer:
                    # print(subseq)
                    kmer[subseq] = kmer[subseq] + 1
                else:
                    kmer[subseq] = 1
            for key, value in kmer.items():
                # print(key)
                # print(value)
                probabilities.append(value/total_windows)
            if e == "Shannon" or e == "shannon":
                entropy_equation = [(p * math.log(p, 2)) for p in probabilities]
                entropy = -(sum(entropy_equation))
                information_entropy.append(entropy)
            else:
                q = 2
                entropy_equation = [(p ** q) for p in probabilities]
                entropy =  (1/(q - 1)) * (1 - sum(entropy_equation))
                information_entropy.append(entropy)
        EntropyClass_file_record(foutput, label_dataset)
    return

        
#############################################################################    
def EntropyClass(finput, foutput):
    ksize = 10
    stepw = 1
    e = 'Shannon'
    label_dataset = 'mRNA'
    entropy_equation(ksize, finput, foutput, e, label_dataset)
       
#############################################################################

In [17]:
# ExtractionTechniques("res.fasta", 'res_1.csv', 'dnc')
# ExtractionTechniques("res.fasta", 'res_1.csv', 'tnc')
# FourierClass("res.fasta", 'res_1.csv', 'mRNA')
# EntropyClass("res.fasta", 'res_1.csv')

Recorded Sequence!!!
Recorded Sequence!!!
Recorded Sequence!!!
Recorded Sequence!!!
Recorded Sequence!!!
